In [1]:
import cv2
import time

In [1]:
import numpy as np
import yagmail

In [3]:
#Set up email credentials (currently, only works with gmail) for yagmail
username = "******"
password = "******"
           
yag = yagmail.SMTP(username, password)

#Depending on whether an elephant or giraffe are detected, respective messages and 
#screenshots are emailed to alert user
elephant_message = ['An elephant has been detected', 'See picture attached' , 
                'Elephant_Detected.jpg']

giraffe_message = ['A giraffe has been detected', 'See picture attached' ,
                'Giraffe_Detected.jpg']

In [4]:
#Load YOLO (neural network module)
net = cv2.dnn.readNet("yolov3.weights","yolo.cfg")

classes = [] #objects that can be detected
with open("coco.names","r") as f:
    classes = [line.strip() for line in f.readlines()]

In [5]:
#Output layer names
layer_names = net.getLayerNames()
outputlayers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]

In [6]:
#Colors for bounding squares depending on object detected
colors = np.random.uniform(0,255,size=(len(classes),3))

In [ ]:
#Set boolean values, so notification email is only sent out once 
elephant_detected = False
giraffe_detected = False

#Load video
#for webcam, pass 0 or respective number
#for demo purposes, using sample file
cap = cv2.VideoCapture("elephants.mp4") 

#Read in dimensions of video or webcam feed
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

#Create cv2 writer object
writer = cv2.VideoWriter('outputFile.mp4', cv2.VideoWriter_fourcc(*'XVID'), 30, (width,height))

font = cv2.FONT_HERSHEY_PLAIN

#Run object detection and drawing of bounding boxes with description
while True:
    
    ret,frame= cap.read()  
    
    if ret == True:
        
        #Optional: rescale and Convert to Gray Scale
        #frame = rescale_frame(frame, percent = 50)
        #frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY )
        #frame = np.stack((frame,)*3, axis=-1)


        height,width,channels = frame.shape
        
        #Create Yolo blob for object detection
        blob = cv2.dnn.blobFromImage(frame,1/255.0,(320,320),(0,0,0),True,crop=False) #reduce to 320    


        net.setInput(blob)
        outs = net.forward(outputlayers)

        
        #Showing info on screen with confidence score of algorithm in detecting an object in blob
        class_ids=[]
        confidences=[]
        boxes=[]
        for out in outs:
            for detection in out:
                scores = detection[5:]
                class_id = np.argmax(scores)

                confidence = scores[class_id]
                if confidence > 0.5:
                    #object detected
                    center_x = int(detection[0]*width)
                    center_y = int(detection[1]*height)
                    w = int(detection[2]*width)
                    h = int(detection[3]*height)

                    
                    #rectangle co-ordinaters
                    x=int(center_x - w/2)
                    y=int(center_y - h/2)
                    
                    
                    boxes.append([x,y,w,h]) #put all rectangle areas
                    confidences.append(float(confidence)) #how confidence was that object detected and show that percentage
                    class_ids.append(class_id) #name of the object that was detected

                    
                    #When elephant is detected, send out notification email with corresponding screenshot
                    if class_id == 20 and elephant_detected == False:
                        print("Elephant Detected")
                        elephant_detected = True
                        cv2.imwrite("Elephant_Detected.jpg", frame)
                        yag.send('to@recipient.com', 'subject', elephant_message)
                        
                        
                    #When giraffe is detected, send out notification email with corresponding screenshot    
                    if class_id == 23 and giraffe_detected == False:
                        print("Giraffe Detected")
                        giraffe_detected = True
                        cv2.imwrite("Giraffe_Detected.jpg", frame)
                        yag.send('to@recipient.com', 'subject', giraffe_message)

                        
        #Draw bounding rectangles on frame/video together with description
        indexes = cv2.dnn.NMSBoxes(boxes,confidences,0.4,0.6)

        for i in range(len(boxes)):
            if i in indexes:
                x,y,w,h = boxes[i]
                label = str(classes[class_ids[i]])
                confidence= confidences[i]
                color = colors[class_ids[i]]
                cv2.rectangle(frame,(x,y),(x+w,y+h),color,2)
                cv2.putText(frame,label+" "+str(round(confidence,2)),(x-10,y+75),font,1,(255,255,255),2)
            
            
        writer.write(frame)

        cv2.imshow("Frame",frame)
        key = cv2.waitKey(1) #wait 1ms the loop will start again and we will process the next frame

        
        if cv2.waitKey(1) & 0xFF == 27:
            break
        
    else:
        break
        
cap.release()    
writer.release()
cv2.destroyAllWindows() 